

- [概率分布间的关系](#概率分布间的关系)
- [LDA](#LDA)
  - [吉布斯抽样法](#吉布斯抽样算法)
  - [变分EM推理](#变分EM推理)
- [算法实现](#算法实现)

# 概率分布间的关系

多项分布是二项分布的扩展，针对多元**离散**随机变量$X$，有 $X \sim \operatorname{Mult}(n, p)$
$$
\begin{aligned} P\left(X_{1}=n_{1}, X_{2}=n_{2}, \cdots, X_{k}=n_{k}\right) &=\frac{n !}{n_{1} ! n_{2} ! \cdots n_{k} !} p_{1}^{n_{1}} p_{2}^{n_{2}} \cdots p_{k}^{n_{k}} \\ &=\frac{n !}{\prod_{i=1}^{k} n_{i} !} \prod_{i=1}^{k} p_{i}^{n_{i}} \end{aligned}
$$
狄利克雷分布是多元**连续**随机变量的概率分布，记作 $\theta \sim \operatorname{Dir}(\alpha)$ 
$$
p(\theta | \alpha)=\frac{\Gamma\left(\sum_{i=1}^{k} \alpha_{i}\right)}{\prod_{i=1}^{k} \Gamma\left(\alpha_{i}\right)} \prod_{i=1}^{k} \theta_{i}^{\alpha_{i}-1}\\
\sum_{i=1}^{k} \theta_{i}=1, \theta_{i} \geqslant 0, \alpha=\left(\alpha_{1}, \alpha_{2}, \cdots, \alpha_{k}\right), \alpha_{i}>0, i=1,2, \cdots, k_{\circ}
$$
$\Gamma(s)$ 是伽玛函数 $\Gamma(s)=\int_{0}^{\infty} x^{s-1} \mathrm{e}^{-x} \mathrm{d} x, \quad s>0$，满足性质 $\Gamma(s+1)=s \Gamma(s)$，当 $s$ 是自然数时，$\Gamma(s+1) = s!$

也可以把狄利克雷分布写成
$$
p(\theta | \alpha)=\frac{1}{\mathrm{B}(\alpha)} \prod_{i=1}^{k} \theta_{i}^{\alpha_{i}-1}\\
\mathrm{B}(\alpha)=\frac{\prod_{i=1}^{k} \Gamma\left(\alpha_{i}\right)}{\Gamma\left(\sum_{i=1}^{k} \alpha_{i}\right)}=\int \prod_{i=1}^{k} \theta_{i}^{\alpha_{i}-1} d \theta
$$
$B(\alpha)$ 是规范化因子，也称多元贝塔函数（贝塔函数的拓展）

二项分布是多项分布的特例，贝塔分布是狄利克雷分布的特例。贝塔分布为
$$
p(x)=\left\{\begin{array}{ll}{\frac{1}{B(s, t)} x^{s-1}(1-x)^{t-1},} & {0 \leqslant x \leqslant 1} \\ {0,} & {其他}\end{array}\right.\\
\mathrm{B}(s, t)=\frac{\Gamma(s) \Gamma(t)}{\Gamma(s+t)}
$$
$B(s,t)$ 为贝塔函数

如果后验分布和先验分布为同类（相同分布族），则先验分布与后验分布成为**共轭分布**，先验成为共轭先验

- 狄利克雷分布属于指数分布族
- 狄利克雷分布是多项分布的共轭先验

> 指数分布族是指概率分布密度能写成如下形式
> $$
> p(x | \eta)=h(x) \exp \left\{\eta^{\mathrm{T}} T(x)-A(\eta)\right\}
> $$
> 多项式分布，泊松分布， gamma 分布，指数分布，beta分布， Dirichlet 分布这些都是一家

![](https://raw.githubusercontent.com/LibertyDream/diy_img_host/master/img/2019-12-11_probability_distribution_relation.png)

# LDA

潜在狄利克雷分配（LDA）是文本集合的生成概率模型，是基于贝叶斯的话题模型。有三要素：**单词集，话题集和文本集**。模型假设

- 话题由单词的多项分布表示（单词产生话题）
- 文本由话题的多项分布表示（话题产生文本）
- 单词分布和话题分布的先验分布都是狄利克雷分布

LDA 是概率图模型，板块表示如下。单词分布、话题分布和文本各位置的话题是隐变量，各位置的单词是观测变量

![](https://raw.githubusercontent.com/LibertyDream/diy_img_host/master/img/2019-12-09_lda-tikz.png)

**文本生成过程**

1. 由参数 $\beta$ 确定的 $Dir(\beta)$，生成若干单词分布（话题） $\varphi_{k} \sim \operatorname{Dir}(\beta)，k=1,2,\ldots,K$
2. 由参数 $\alpha$ 确定的 $Dir(\alpha)$，生成若干话题分布（文本）$\theta_{m} \sim \operatorname{Dir}(\alpha)，m=1,2,\ldots,M$
3. 对于文本 $\mathbf{w}_m$ 里的单词 $w_{mn},n=1,2\ldots,N$
   1. 生成话题 $z_{m n} \sim \operatorname{Mult}\left(\theta_{m}\right)$
   2. 生成单词 $w_{m n} \sim \operatorname{Mult}\left(\varphi_{z_{m n}}\right)$

话题个数 $K$ 通常由实验选定。狄利克雷分布的参数 $\alpha,\beta$ 也是事先给定。没有先验知识时所有分量默认为 1

LDA 模型本身的学习与推理不能直接求解。通常采用吉布斯抽样和变分 EM 算法，前者是蒙特卡罗法（MCMC），通过抽样近似估计，简单但迭代次数多。后者是解析计算进行近似，推理和学习效率高

## 吉布斯抽样算法

超参数已给定，目标是对联合概率分布 $p(\mathbf{w},\mathbf{z},\theta,\varphi|\alpha,\beta)$ 进行估计，$\mathbf{w}$ 已知，$\mathbf{z},\theta,\varphi$ 未知。通过积分求和将隐变量 $\theta$ 和 $\varphi$ 消掉，得到边缘分布 $p(\mathbf{w},\mathbf{z}|\alpha,\beta)$。对概率分布 $p(\mathbf{w}|\alpha,\beta,\mathbf{z})$ 进行吉布斯抽样，得到该分布下的随机样本，利用样本对 $\mathbf{z},\theta,\varphi$ 进行估计。最终得到  $p(\mathbf{w},\mathbf{z},\theta,\varphi|\alpha,\beta)$ 的参数估计。

1. 对给定文本的单词序列 $\mathbf{w}$，每个位置随机指派一个话题，构成整个文本的话题序列 $\mathbf{z}$

2. 循环直到燃烧期

   1. 每个位置上计算该处话题的满条件概率分布，进行随机抽样，得到该位置的新话题

   $$
   p\left(z_{i} | \mathbf{z}_{-i}, \mathbf{w}, \alpha, \beta\right) \propto \frac{n_{k v}+\beta_{v}}{\sum_{v=1}^{V}\left(n_{k v}+\beta_{v}\right)} \cdot \frac{n_{m k}+\alpha_{k}}{\sum_{k=1}^{K}\left(n_{m k}+\alpha_{k}\right)}
   $$

   前一项是由话题生成单词的概率，后一项是由文本生成话题的概率

   2. 整体准备两个计数矩阵：话题-单词矩阵 $N_{K \times V}=\left[n_{k v}\right]$ 和文本-话题矩阵 $N_{M \times K}=[n_{mk}]$。两个矩阵里该位置旧话题计数减 1，计算满条件概率，抽样，得到该处新话题，该处新话题计数加 1。前往下一个位置

3. 燃烧期后所有文本的话题序列就是后验概率分布  $p(\mathbf{z}|\alpha,\beta,\mathbf{w})$ 的样本计数，由此计算模型参数

$$
\begin{aligned} \theta_{m k}=& \frac{n_{m k}+\alpha_{k}}{\sum_{k=1}^{K}\left(n_{m k}+\alpha_{k}\right)} \\ \varphi_{k v}=& \frac{n_{k v}+\beta_{v}}{\sum_{v=1}^{V}\left(n_{k v}+\beta_{v}\right)} \end{aligned}
$$

## 变分EM推理

假设模型是联合概率分布 $p(x,z)$，$x$ 是观测变量（数据），$z$ 是隐变量。目标是学习模型的后验概率分布 $p(z|x)$。考虑用变分分布 $q(z)$ 近似条件概率分布 $p(z|x)$，用 KL 散度计算二者相似性找到与 $p(z|x)$ 在 KL 散度意义下最近的 $q^*(z)$，用该分布近似 $p(z|x)$。

> KL 散度是对两个概率分布相似性的一种度量。记作 $D(Q||P)$，对离散随机变量和连续随机变量有
> $$
> D(Q \| P)=\sum_{i} Q(i) \log \frac{Q(i)}{P(i)}\\
> D(Q \| P)=\int Q(x) \log \frac{Q(x)}{P(x)} \mathrm{d} x
> $$
> 由 Jesson 不等式
> $$
> \begin{aligned}-D(Q \| P) &=\int Q(x) \log \frac{P(x)}{Q(x)} \mathrm{d} x \\ & \leqslant \log \int Q(x) \frac{P(x)}{Q(x)} \mathrm{d} x \\ &=\log \int P(x) \mathrm{d} x=0 \end{aligned}
> $$
> 所以 KL 散度具有性质 $D(Q||P) \geqslant 0$ ，当且仅当两分布相同才为 0。
>
> KL 散度不对称，也不满足三角不等式，不是传统意义的距离度量

对变分分布要求容易处理，所以假设 $q(z)$ 中 $z$ 的所有分量都相互独立，此时变分分布称为平均场。利用 Jensen 不等式，得到 KL 散度最小化可以通过证据下界最大化实现。变分推理由此变成下界最大化问题
$$
\begin{aligned} D(q(z) \| p(z | x)) &=E_{q}[\log q(z)]-E_{q}[\log p(z | x)] \\ &=E_{q}[\log q(z)]-E_{q}[\log p(x, z)]+\log p(x) \\ &=\log p(x)-\{E_{q}[\log p(x, z)]-E_{q}[\log q(z)]\}\end{aligned}\\
L(q, \theta)=E_{q}[\log p(x, z | \theta)]-E_{q}[\log q(z)]
$$
针对LDA模型定义变分分布，应用 EM 变分法。
$$
q(\theta, \mathbf{z} | \gamma, \eta)=q(\theta | \gamma) \prod_{n=1}^{N} q\left(z_{n} | \eta_{n}\right)
$$
一共 N 个词，$\gamma$ 是狄利克雷分布的参数，$\eta$ 是多项分布参数

目标是对下界 $L(\gamma, \eta, \alpha, \varphi)$ 最大化，$\alpha,\varphi$ 是LDA 模型参数，$\gamma,\eta$ 是变分参数。
$$
\begin{aligned}
L(\gamma, \eta, \alpha, \varphi)&=E_{q}[\log p(\theta, \mathbf{z}, \mathbf{w} | \alpha, \varphi)]-E_{q}[\log q(\theta, \mathbf{z} | \gamma, \eta)]\\
 &= E_{q}[\log p(\theta | \alpha)]+E_{q}[\log p(\mathbf{z} | \theta)]+E_{q}[\log p(\mathbf{w} | \mathbf{z}, \varphi)]-E_{q}[\log q(\theta | \gamma)]-E_{q}[\log q(\mathbf{z} | \eta)] 
 \end{aligned}
$$
$\theta$ 为话题分布，交替 E，M 两步，直到收敛

1. E 步：固定参数 $\alpha,\varphi$ ，通过变分参数 $\gamma,\eta$ 证据下界最大化估计 $\gamma,\eta$ 
2. M步：固定 $\gamma,\eta$ ，通过模型参数 $\alpha,\varphi$ 证据下界最大化估计 $\alpha,\varphi$ 

# 算法实现

**导入相关库**

In [1]:
import numpy as np

**硬件与版本信息**

In [2]:
%load_ext watermark
%watermark -m -v -p ipywidgets,numpy

CPython 3.7.3
IPython 7.6.1

ipywidgets 7.5.0
numpy 1.16.4

compiler   : MSC v.1915 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 60 Stepping 3, GenuineIntel
CPU cores  : 4
interpreter: 64bit


**LDA 实现**

In [3]:
class LDA(object):
    
    def __init__(self, T, alpha=None, beta=None):
        '''基于吉布斯抽样的潜在狄利克雷分配

        Parameters：
        -----------------
        T:
            话题数
        alpha：
            先验话题狄利克雷分布参数
        beta：
            先验单词狄利克雷分布参数

        Attributes：
        --------------------
        D:
            文本数
        N:
            单词数
        V:
            单词标识数，一个标识唯一对应一个单词
        phi：
            单词分布
        theta：
            话题分布
        '''
        self.T = T
        
        if alpha is None:
            self.alpha = (50.0 / self.T) * np.ones(self.T)
        else:
            self.alpha = alpha * np.ones(self.T)
        
        if beta is None:
            self.beta = 0.01
        else:
            self.beta = beta
            
    def __init_params(self, texts, tokens):
        
        self.tokens = tokens
        self.D = len(texts)
        self.V = len(np.unique(self.tokens))
        self.N = np.sum(np.array([len(doc) for doc in texts]))
        self.word_document = np.zeros(self.N)
        
        self.beta = self.beta * np.ones(self.V)
        
        count = 0
        for doc_idx, doc in enumerate(texts):
            for word_idx, word in enumerate(doc):
                word_idx = word_idx + count
                self.word_document[word_idx] = doc_idx
            count += len(doc)
    
    def train(self, texts, tokens, burn_limits=2000):
        '''在给定语料和标识集上训练 LDA 模型
        
        Returns
        ---------------------
        C_wt：
            单词-话题计数矩阵
        C_dt：
            文本-话题计数矩阵
        assignments:
            抽样时分配给每个单词位置的话题
        '''
        
        self.__init_params(texts, tokens)
        C_wt, C_dt, assignments = self.__gibbs_sampler(burn_limits, texts)
        self.fit_params(C_wt, C_dt)
        return C_wt, C_dt, assignments
    
    def topic_words_distribution(self, top_n=10):
        '''按类别输出各话题下 Top-N 个单词'''
        for t in range(self.T):
            idx = np.argsort(self.phi[:,t])[::-1][:top_n]
            tokens = self.tokens[idx]
            for token in tokens:
                print('%s\n' % (str(token)))
                
    def fit_params(self, C_wt, C_dt):
        '''由抽样结果计算参数估计值'''
        
        self.phi = np.zeros([self.V, self.T])
        self.theta = np.zeros([self.D, self.T])
        
        b,a = self.beta[0], self.alpha[0]
        for i in range(self.V):
            for j in range(self.T):
                self.phi[i,j] = (C_wt[i, j] + b) / (np.sum(C_wt[:,j]) + self.V * b)
        
        for i in range(self.D):
            for j in range(self.T):
                self.theta[i,j] = (C_dt[i, j] + a) / (np.sum(C_dt[i,:]) + self.T * a)
    
        return self.phi, self.theta
    
    def __full_condition_prob(self, i, d, C_wt, C_dt):
        '''计算第 i 个位置的满条件概率'''
        
        p_vec = np.zeros(self.T)
        b, a = self.beta[0], self.alpha[0]
        for j in range(self.T):
            frac1 = (C_wt[i, j] + b) / (np.sum(C_wt[:, j])+self.V * b)
            frac2 = (C_dt[d, j] + a) / (np.sum(C_dt[d, :])+self.T * a)
            p_vec[j] = frac1 * frac2
        return p_vec / np.sum(p_vec)
    
    def __gibbs_sampler(self, burn_limits, texts):
        '''吉布斯采样估计后验概率'''
        
        C_wt = np.zeros([self.V, self.T])
        C_dt = np.zeros([self.D, self.T])
        assignments = np.zeros([self.N, burn_limits + 1])
        
        for i in range(self.N):
            token_idx = np.concatenate(texts)[i]
            assignments[i, 0] = np.random.randint(0, self.T)
            
            doc = self.word_document[i]
            C_dt[doc, assignments[i, 0]] += 1
            C_wt[token_idx, assignments[i, 0]] += 1
            
        for i in range(burn_limits):
            print('iteration %d of %d' % (i+1, burn_limits))
            for j in range(self.N):
                token_idx = np.concatenate(texts)[j]
                
                doc = self.word_document[j]
                C_wt[token_idx, assignments[j, i]] -= 1
                C_dt[doc, assignments[j, i]] -= 1
                
                p_topic = self.__full_condition_prob(token_idx, doc, C_wt, C_dt)
                sample = np.nonzero(np.random.multinomial(1, p_topic))[0][0]
                
                C_wt[token_idx, sample] += 1
                C_dt[doc, sample] += 1
                assignments[j, i+1] = sample
            
        return C_wt, C_dt, assignments

---

**作者：** Daniel Meng

**GitHub：**[LibertyDream](https://github.com/LibertyDream)

**博客：**[明月轩](https://LIbertydream.github.io)